In [1]:
from keras.layers import Conv2D, Flatten, MaxPool2D, Dense, BatchNormalization
from keras.layers import UpSampling2D, Conv2DTranspose, Reshape
from keras.models import Input, Model, Sequential, load_model
from keras.layers import LeakyReLU, Activation
from keras.datasets.mnist import load_data
import numpy as np
import h5py

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
def generator():

    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(Activation('tanh'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

    return gen

In [12]:
def discriminator():
    
    model = Sequential()
    model.add(Conv2D(64, (5, 5), padding='same', input_shape=(28, 28, 1)))
    model.add(Activation('tanh'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

    return dis

In [13]:
def GAN(gen, dis):
    
    model = Sequential()
    model.add(gen)
    dis.trainable = False
    model.add(dis)
    
    return model

In [14]:
(x_train, _), (x_test, _) = load_data()

x_train = x_train/255
x_test = x_test/255

x_train = x_train[:, :, :, None]
x_test = x_test[:, :, :, None]

gen = generator()
dis = discriminator()

genAdvNet = GAN(gen, dis)

gen.compile(loss='binary_crossentropy', optimizer='adadelta')
genAdvNet.compile(loss='binary_crossentropy', optimizer='adadelta')
dis.trainable = True
dis.compile(loss='binary_crossentropy', optimizer='adadelta')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  after removing the cwd from sys.path.


In [ ]:
y = np.concatenate((np.ones(512), np.zeros(512)))
z = np.ones(512)
for epoch in range(100):
    for i in range(60000//512):
        
        noise = np.random.uniform(-1, 1, size=(512, 100))
        img = x_train[i*512:(i+1)*512]
        
        pred = gen.predict(noise)
        
        X = np.concatenate((img, pred))
        
        
        d_loss = dis.train_on_batch(X, y)
        
        
        dis.trainable = False
        noise = np.random.uniform(-1, 1, size=(512, 100))
        
        g_loss = genAdvNet.train_on_batch(noise, z)
        dis.trainable = True
        
    print("batch %d d_loss : %f" % (d_loss))
    print("batch %d g_loss : %f" % (g_loss))